In [2]:
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero , HeteroConv , GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score , matthews_corrcoef

import TropiGAT_functions 
#from TropiGAT_functions import get_top_n_kltypes ,clean_print 

import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
import logging
from multiprocessing.pool import ThreadPool
warnings.filterwarnings("ignore")

# *****************************************************************************
# Load the Dataframes :
path_work = "/media/concha-eloko/Linux/PPT_clean"
path_ensemble = f"{path_work}/ficheros_28032023/winning_ensemble_0702"
#path_ensemble = f"{path_work}/ficheros_28032023/ensemble_tailored_0612"


> Make model : 

In [3]:
dico_models, errors = TropiGAT_functions.make_ensemble_TropiGAT(path_ensemble)

In [32]:
dico_t = {'KL1': {'optimal_MCC': 0.795242772487544, 'optimal_t': 0.23232323232323235},
 'KL10': {'optimal_MCC': 0.7818181818181819, 'optimal_t': 0.595959595959596},
 'KL102': {'optimal_MCC': 0.8733335395405587, 'optimal_t': 0.686868686868687},
 'KL105': {'optimal_MCC': 0.9452073465116987, 'optimal_t': 0.696969696969697},
 'KL106': {'optimal_MCC': 0.8175155958303386, 'optimal_t': 0.5353535353535354},
 'KL107': {'optimal_MCC': 0.8755668091455752, 'optimal_t': 0.6767676767676768},
 'KL108': {'optimal_MCC': 0.8366600265340756, 'optimal_t': 0.6565656565656566},
 'KL110': {'optimal_MCC': 1.0, 'optimal_t': 0.22222222222222224},
 'KL111': {'optimal_MCC': 0.9116846116771036, 'optimal_t': 0.7373737373737375},
 'KL112': {'optimal_MCC': 0.8980265101338746, 'optimal_t': 0.42424242424242425},
 'KL114': {'optimal_MCC': 0.7817359599705717, 'optimal_t': 0.686868686868687},
 'KL116': {'optimal_MCC': 1.0, 'optimal_t': 0.16161616161616163},
 'KL118': {'optimal_MCC': 1.0, 'optimal_t': 0.48484848484848486},
 'KL12': {'optimal_MCC': 0.7302967433402215, 'optimal_t': 0.37373737373737376},
 'KL122': {'optimal_MCC': 0.6831300510639733, 'optimal_t': 0.9090909090909092},
 'KL123': {'optimal_MCC': 0.6546536707079772, 'optimal_t': 0.6363636363636365},
 'KL125': {'optimal_MCC': 0.6813851438692469, 'optimal_t': 0.8989898989898991},
 'KL127': {'optimal_MCC': 0.5443310539518174, 'optimal_t': 0.8181818181818182},
 'KL128': {'optimal_MCC': 0.778498944161523, 'optimal_t': 0.7676767676767677},
 'KL13': {'optimal_MCC': 0.7784989441615229, 'optimal_t': 0.696969696969697},
 'KL136': {'optimal_MCC': 0.8385254915624211, 'optimal_t': 0.5050505050505051},
 'KL14': {'optimal_MCC': 0.7627700713964739, 'optimal_t': 0.9494949494949496},
 'KL145': {'optimal_MCC': 1.0, 'optimal_t': 0.4747474747474748},
 'KL149': {'optimal_MCC': 0.8563488385776753, 'optimal_t': 0.3434343434343435},
 'KL15': {'optimal_MCC': 0.8288922659741738, 'optimal_t': 0.6060606060606061},
 'KL151': {'optimal_MCC': 0.8, 'optimal_t': 0.4545454545454546},
 'KL16': {'optimal_MCC': 0.7844645405527361, 'optimal_t': 0.38383838383838387},
 'KL169': {'optimal_MCC': 0.6504436355879909, 'optimal_t': 0.26262626262626265},
 'KL17': {'optimal_MCC': 0.9628839072479044, 'optimal_t': 0.6767676767676768},
 'KL18': {'optimal_MCC': 0.6396021490668313, 'optimal_t': 0.36363636363636365},
 'KL19': {'optimal_MCC': 0.9223309842157773, 'optimal_t': 0.5656565656565657},
 'KL2': {'optimal_MCC': 0.8378688987532451, 'optimal_t': 0.43434343434343436},
 'KL21': {'optimal_MCC': 0.828302688169126, 'optimal_t': 0.8181818181818182},
 'KL22': {'optimal_MCC': 1.0, 'optimal_t': 0.5252525252525253},
 'KL23': {'optimal_MCC': 0.8570333046693909, 'optimal_t': 0.9393939393939394},
 'KL24': {'optimal_MCC': 0.8011713970092863, 'optimal_t': 0.4444444444444445},
 'KL25': {'optimal_MCC': 0.8482467449833488, 'optimal_t': 0.48484848484848486},
 'KL27': {'optimal_MCC': 0.8584556465075666, 'optimal_t': 0.7575757575757577},
 'KL28': {'optimal_MCC': 0.7824675324675324, 'optimal_t': 0.7171717171717172},
 'KL29': {'optimal_MCC': 0.6454972243679029, 'optimal_t': 0.5858585858585859},
 'KL3': {'optimal_MCC': 0.8570693678078618, 'optimal_t': 0.9797979797979799},
 'KL30': {'optimal_MCC': 0.7342995169082126, 'optimal_t': 0.6464646464646465},
 'KL34': {'optimal_MCC': 0.7745966692414834, 'optimal_t': 0.43434343434343436},
 'KL36': {'optimal_MCC': 0.7386222109866251, 'optimal_t': 0.8181818181818182},
 'KL38': {'optimal_MCC': 0.6733804983414345, 'optimal_t': 0.36363636363636365},
 'KL39': {'optimal_MCC': 0.7947194142390263, 'optimal_t': 0.22222222222222224},
 'KL43': {'optimal_MCC': 0.6396021490668313, 'optimal_t': 0.36363636363636365},
 'KL45': {'optimal_MCC': 0.8376233659741151, 'optimal_t': 0.7777777777777778},
 'KL46': {'optimal_MCC': 0.85, 'optimal_t': 0.8282828282828284},
 'KL47': {'optimal_MCC': 0.9670913609804876, 'optimal_t': 0.4747474747474748},
 'KL48': {'optimal_MCC': 0.6396021490668313, 'optimal_t': 0.5555555555555556},
 'KL5': {'optimal_MCC': 0.6546536707079772, 'optimal_t': 0.42424242424242425},
 'KL51': {'optimal_MCC': 0.8883781212268286, 'optimal_t': 0.6161616161616162},
 'KL52': {'optimal_MCC': 0.7934920476158722, 'optimal_t': 0.9292929292929294},
 'KL53': {'optimal_MCC': 0.6831300510639732, 'optimal_t': 0.7171717171717172},
 'KL55': {'optimal_MCC': 0.7817359599705717, 'optimal_t': 0.17171717171717174},
 'KL57': {'optimal_MCC': 0.76, 'optimal_t': 0.6262626262626263},
 'KL60': {'optimal_MCC': 0.7276068751089989, 'optimal_t': 0.8484848484848485},
 'KL62': {'optimal_MCC': 0.7459688097630196, 'optimal_t': 0.9090909090909092},
 'KL63': {'optimal_MCC': 1.0, 'optimal_t': 0.98989898989899},
 'KL64': {'optimal_MCC': 0.8967653421507276, 'optimal_t': 0.8181818181818182},
 'KL7': {'optimal_MCC': 0.8416254115301732, 'optimal_t': 0.9090909090909092},
 'KL70': {'optimal_MCC': 0.7905694150420948, 'optimal_t': 0.787878787878788},
 'KL74': {'optimal_MCC': 0.7664191894673175, 'optimal_t': 0.6161616161616162},
 'KL8': {'optimal_MCC': 0.7745966692414834, 'optimal_t': 0.5151515151515152},
 'KL81': {'optimal_MCC': 0.5773502691896257, 'optimal_t': 0.6565656565656566},
 'KL9': {'optimal_MCC': 1.0, 'optimal_t': 0.6161616161616162}}


@torch.no_grad()
def make_predictions(model, data):
	model.eval() 
	output = model(data)
	probabilities = torch.sigmoid(output)
	predictions = probabilities.round() 
	return predictions, round(probabilities.item() , 4) 
    
def run_prediction_optimized(query_graph, dico_ensemble, dico_t) :
    dico_predictions = {}
    for KL_type in dico_ensemble :
        model = dico_ensemble[KL_type]
        prediction, probabilities = make_predictions(model, query_graph)
        if probabilities >= dico_t[KL_type]["optimal_t"] :
            dico_predictions[KL_type] = probabilities
        else :
            continue
    return dico_predictions

def run_prediction_optimized_v1(query_graph, dico_ensemble, dico_t) :
    dico_predictions = {}
    for KL_type in dico_ensemble :
        model = dico_ensemble[KL_type]
        prediction, probabilities = make_predictions(model, query_graph)
        if probabilities >= dico_t[KL_type]["optimal_t"] :
            s = (probabilities - dico_t[KL_type]["optimal_t"])/(probabilities + dico_t[KL_type]["optimal_t"])
            a = {"probability" : probabilities , "threshold" : dico_t[KL_type]["optimal_t"] , "score" : s}
            dico_predictions[KL_type] = a
        else :
            continue
    return dico_predictions

In [25]:
(0.9 - 0.7) / (0.9 + 0.7) , (0.9 - 0.4) / (0.9 + 0.4)

(0.12500000000000003, 0.3846153846153846)

> Ferriol

In [4]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/77_strains_phage_project"
path_Dpo_domain_org = "/media/concha-eloko/Linux/depolymerase_building/clean_77_phages_depo"

dpo_embeddings = pd.read_csv(f"{path_project}/rbp_work/Dpo_domains_77.esm2.embedding.1512.csv", sep = "," , header = None)
dpo_embeddings = dpo_embeddings.drop([1281] , axis = 1)
dpo_embeddings.set_index([0], inplace = True)
dpo_embeddings.index = [x if x.count("__cds") > 0 else x.replace("_cds", "__cds") for x in dpo_embeddings.index]
dpo_embeddings.index = [x.replace("__CDS","__cds") if x.count("__CDS") > 0 else x for x in dpo_embeddings.index]
dpo_embeddings.index = ['__'.join(x.split("__")[0:-1])+'_Dpo_domain' if x.count("_Dpo_domain") == 0 else x for x in dpo_embeddings.index]

> Bea

In [5]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

bea_embeddings = pd.read_csv(f"{path_project}/Bea_phages.esm2.embedding.csv", sep = "," , header = None)
bea_embeddings = bea_embeddings.drop([1281] , axis = 1)
bea_embeddings.set_index([0], inplace = True)


> Towndsend

In [6]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

towndsend_embeddings = pd.read_csv(f"{path_project}/Townsed_phages.esm2.embedding.1112.csv", sep = "," , header = None)
towndsend_embeddings = towndsend_embeddings.drop([1281] , axis = 1)
towndsend_embeddings.set_index([0], inplace = True)


***
> run the predictions 

In [33]:
# Run the predictions ferriol :
ferriol_predictions = {}
for dpo in dpo_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([dpo_embeddings.loc[dpo].values])
    pred = run_prediction_optimized(graph_dpo,dico_models,dico_t)
    ferriol_predictions[dpo] = pred

In [36]:
ferriol_predictions

{'K15PH90__cds_55_Dpo_domain': {'KL34': 0.6161,
  'KL60': 0.9957,
  'KL27': 0.9988,
  'KL43': 0.9609,
  'KL15': 0.9951,
  'KL3': 0.9981,
  'KL145': 0.978,
  'KL116': 0.2347,
  'KL13': 0.983,
  'KL51': 0.703,
  'KL123': 0.7221,
  'KL118': 0.7247,
  'KL18': 0.9927},
 'K80PH1317b__cds_54_Dpo_domain': {'KL1': 0.3308,
  'KL60': 0.8626,
  'KL2': 0.4854,
  'KL128': 0.8025},
 'K64PH164C4__cds_24_Dpo_domain': {'KL57': 0.8393,
  'KL38': 0.5179,
  'KL60': 0.8576,
  'KL45': 0.8555,
  'KL64': 0.9791,
  'KL13': 0.9633,
  'KL14': 0.9952,
  'KL18': 0.8389,
  'KL169': 0.2826},
 'K5lambda5__cds_196_Dpo_domain': {'KL25': 0.9032,
  'KL57': 0.762,
  'KL60': 0.995,
  'KL39': 0.4264,
  'KL145': 0.7319,
  'KL149': 0.56,
  'KL123': 0.7767,
  'KL18': 0.9977},
 'K11PH164C1__cds_46_Dpo_domain': {'KL25': 0.4888,
  'KL57': 0.6671,
  'KL74': 0.6347,
  'KL43': 0.6545,
  'KL145': 0.8115,
  'KL123': 0.9221},
 'K57lambda1_2__cds_93_Dpo_domain': {'KL25': 0.7012,
  'KL34': 0.6528,
  'KL48': 0.7114,
  'KL74': 0.9766,
  'KL

***

In [34]:
# Run the predictions Bea :
bea_predictions = {}
for dpo in bea_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([bea_embeddings.loc[dpo].values])
    pred = run_prediction_optimized(graph_dpo,dico_models,dico_t)
    bea_predictions[dpo] = pred

In [35]:
# Run the predictions Towndsend :
towndsend_predictions = {}
for dpo in towndsend_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([towndsend_embeddings.loc[dpo].values])
    pred = run_prediction_optimized(graph_dpo,dico_models,dico_t)
    towndsend_predictions[dpo] = pred

***
# Write the results : 

> Predictions : 

In [37]:
predictions = [ferriol_predictions , bea_predictions , towndsend_predictions]

with open("/media/concha-eloko/Linux/PPT_clean/TropiGAT.results.classic_2002.optimized.tsv", "w") as outfile:
    for prediction in predictions:
        prediction_sorted = dict(sorted(prediction.items()))
        for prot in prediction_sorted:
            if prediction_sorted[prot] == "No hits" or len(prediction_sorted[prot]) == 0:
                outfile.write(f"{prot}\tNo hits\n")
            else:
                outfile.write(f"{prot}\t")
                hits = [f"{kltype}:{round(score, 3)}" for kltype, score in prediction_sorted[prot].items()]
                sorted_hits = " ; ".join(sorted(hits, key=lambda x: float(x.split(":")[1]), reverse=True))
                outfile.write(sorted_hits)
                outfile.write("\n")

> Parse TropiGAT and Seqbased results :

In [38]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean"

tropigat_results = pd.read_csv(f"{path_project}/TropiGAT.results.classic_2002.optimized.tsv", sep = "\t", names = ["protein", "predictions_tropigat"])

seqbased_results = pd.read_csv(f"{path_project}/Seqbased_model.results.bit75.1902.tsv", sep = "\t", names = ["protein", "predictions_seqbased"])


In [39]:
tropigat_results["protein_id"] = tropigat_results["protein"].apply(lambda x : x.split("_Dpo")[0])
seqbased_results["protein_id"] = seqbased_results["protein"].apply(lambda x : x.split("_A")[0] if x.count("_A")>0 else "__".join(x.split(",")[0].split(" ")) if x.count(",")>0 else x)

merged_df = tropigat_results.merge(seqbased_results, on='protein_id', how='inner')
merged_df["phage"] = merged_df["protein_id"].apply(lambda x : x.split("__")[0] if x.count("__")>0 else x.split("_")[0])

merged_df_sorted = merged_df.sort_values(by='phage', ascending=True)
merged_df_sorted

,protein_x,predictions_tropigat,protein_id,protein_y,predictions_seqbased,phage
123,A1a_00002,KL123:0.977 ; KL13:0.9 ; KL39:0.881 ; KL114:0....,A1a_00002,A1a_00002,KL102:0.737,A1a
124,A1a_00014,KL45:0.878 ; KL2:0.741 ; KL116:0.454 ; KL110:0...,A1a_00014,A1a_00014,KL151:0.599,A1a
125,A1b_00036,KL7:0.983 ; KL13:0.975 ; KL123:0.971 ; KL118:0...,A1b_00036,A1b_00036,KL102:0.737,A1b
126,A1b_00048,KL128:0.919 ; KL70:0.885 ; KL28:0.751 ; KL38:0...,A1b_00048,A1b_00048,No predictions,A1b
127,A1c_00034,KL52:0.998 ; KL62:0.993 ; KL128:0.981 ; KL43:0...,A1c_00034,A1c_00034,No hits,A1c
...,...,...,...,...,...,...
189,S13a_00036,KL18:0.959 ; KL27:0.938 ; KL60:0.87 ; KL123:0....,S13a_00036,S13a_00036,No predictions,S13a
190,S13b_00058,KL47:0.988 ; KL43:0.633 ; KL24:0.582 ; KL39:0.282,S13b_00058,S13b_00058,KL63:0.641,S13b
191,S13c_00055,KL27:0.958 ; KL38:0.955 ; KL60:0.92 ; KL57:0.9...,S13c_00055,S13c_00055,No predictions,S13c
192,S13d_00057,KL14:0.999 ; KL52:0.973 ; KL38:0.954 ; KL27:0....,S13d_00057,S13d_00057,KL14:0.951,S13d


In [40]:
final_df = merged_df_sorted[["phage","protein_id", "predictions_seqbased", "predictions_tropigat"]]



In [41]:
final_df.to_csv(f"{path_project}/PPT_results.classic_2002_optimized.bit75.tsv", sep = "\t", header = True, index = False)

In [42]:
final_df

,phage,protein_id,predictions_seqbased,predictions_tropigat
123,A1a,A1a_00002,KL102:0.737,KL123:0.977 ; KL13:0.9 ; KL39:0.881 ; KL114:0....
124,A1a,A1a_00014,KL151:0.599,KL45:0.878 ; KL2:0.741 ; KL116:0.454 ; KL110:0...
125,A1b,A1b_00036,KL102:0.737,KL7:0.983 ; KL13:0.975 ; KL123:0.971 ; KL118:0...
126,A1b,A1b_00048,No predictions,KL128:0.919 ; KL70:0.885 ; KL28:0.751 ; KL38:0...
127,A1c,A1c_00034,No hits,KL52:0.998 ; KL62:0.993 ; KL128:0.981 ; KL43:0...
...,...,...,...,...
189,S13a,S13a_00036,No predictions,KL18:0.959 ; KL27:0.938 ; KL60:0.87 ; KL123:0....
190,S13b,S13b_00058,KL63:0.641,KL47:0.988 ; KL43:0.633 ; KL24:0.582 ; KL39:0.282
191,S13c,S13c_00055,No predictions,KL27:0.958 ; KL38:0.955 ; KL60:0.92 ; KL57:0.9...
192,S13d,S13d_00057,KL14:0.951,KL14:0.999 ; KL52:0.973 ; KL38:0.954 ; KL27:0....
